In [68]:
import os
from pprint import *
import pandas as pd
import numpy as np
import seaborn as sns
from dateutil import parser
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,8.27)})
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from tqdm import *
from pathlib import Path


In [10]:
channels = pd.read_csv('../csv/channels.csv', index_col=0)
videos = pd.read_csv('../csv/videos.csv', index_col = 0, sep =';')
channels = channels[channels.channelId != "UCCCPCZNChQdGa9EkATeye4g"]
videos = videos[videos.channelId != "UCCCPCZNChQdGa9EkATeye4g"]
videos_m = pd.merge(videos, channels, left_on="channelId", right_on="channelId", how="left")
videos_m = videos_m.rename(columns={
    'publishedAt_x': 'publishedAt_video',
    'title_x': 'title_video',
    'viewCount_x': 'viewCount_video',
    'title_y': 'title_channel',
    'viewCount_y': 'viewCount_channel'
}
)
videos_m = videos_m.assign(
    #engagement_1 = (((videos_m.likeCount + videos_m.commentCount) / videos_m.videoCount) / videos_m.subscriberCount) * 100,
    engagement = ((videos_m.likeCount + videos_m.commentCount) / videos_m.subscriberCount) * 100,
    #engagement_3 = (((videos_m.likeCount + videos_m.commentCount + videos_m.viewCount_video) / videos_m.videoCount) / videos_m.subscriberCount) * 100,
    #engagement_4 = ((videos_m.likeCount + videos_m.commentCount + videos_m.viewCount_video) / videos_m.subscriberCount) * 100,
    #engagement_5 = (videos_m.viewCount_video + videos_m.likeCount + videos_m.commentCount) / videos_m.subscriberCount
    
) 
videos = videos_m

In [11]:
def removing_numbers(text):
    return ''.join([i for i in text if not i.isdigit()])
import emoji
import regex

def is_emoji(s):
    return s in UNICODE_EMOJI

def extract_emojis(text):
    
    emoji_list = []
    data = regex.findall(r'\X', text)
    flags = regex.findall(u'[\U0001F1E6-\U0001F1FF]', text) 
    data = text.split()
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return len(emoji_list )#+ flags)

def number_caplock_words(text):
    text = text.replace("’", " ")
    text = text.replace("'", " ")
    text = text.replace(",", " ")
    text = text.replace("!", " ")
    text = text.replace("?", " ")
    text = text.replace(".", " ")
    text = text.replace("/", " ")
    text = text.replace("\n", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.split()
    text
    c = 0
    for word in text:
        #print(word)
        if word.isupper():
            c += 1
    return c 

from collections import Counter
import re

def number_hastags(text):
    lst = [text]
    hash_counts = Counter(re.findall(r'#([a-z0-9]+)', ' '.join(lst), re.I))
    return len(dict(hash_counts).keys())

def number_punctuation(text):
    count = 0;  
    str = text  
    for i in range (0, len (str)):   
        #Checks whether given character is a punctuation mark  
        if str[i] in ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?"):  
            count = count + 1
    return count

def number_diff_words(text):
    list_of_words = text.split()
    list_of_words = [''.join(e for e in word if e.isalnum()) for word in list_of_words]
    list_of_words = [word for word in list_of_words if word]
    list_of_words = list(set(list_of_words))
    return len(list_of_words)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def number_sentances(text):
    number_of_sentences = sent_tokenize(text)

    return len(number_of_sentences)

def number_words(text):
    return len(text.split())

import re
def remove_emojis(data):
    data = data.replace("\n", " ")
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package punkt to /home/romain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
videos_m

,channelId,videoId,tn_url,publishedAt_video,publishedYear,publishedMonth,publishedDay,publishedWeekday,publishedHour,title_video,...,color_2_b,color_3_r,color_3_g,color_3_b,title_channel,publishedAt_y,viewCount_channel,subscriberCount,videoCount,engagement
0,UCYpRDnhk5H8h16jpS84uqsA,6_o8EwK-m7o,https://i.ytimg.com/vi/6_o8EwK-m7o/default.jpg,2022-06-03T16:16:46Z,2022.0,6.0,3.0,4.0,18.0,🔴 Stade de France : comment l’organisation de ...,...,158.0,150.0,150.0,145.0,Le Monde,2006-03-17T14:41:41Z,310639457,1320000,2055,0.168712
1,UCYpRDnhk5H8h16jpS84uqsA,YKeP0l1VGzE,https://i.ytimg.com/vi/YKeP0l1VGzE/default.jpg,2022-06-02T15:14:37Z,2022.0,6.0,2.0,3.0,17.0,Jubilé d'Elizabeth II : la reine au balcon de ...,...,205.0,184.0,183.0,191.0,Le Monde,2006-03-17T14:41:41Z,310639457,1320000,2055,0.091439
2,UCYpRDnhk5H8h16jpS84uqsA,NeFjybzGbzc,https://i.ytimg.com/vi/NeFjybzGbzc/default.jpg,2022-05-29T12:00:09Z,2022.0,5.0,29.0,6.0,14.0,Pourquoi Molière est devenu célèbre ?,...,203.0,133.0,144.0,154.0,Le Monde,2006-03-17T14:41:41Z,310639457,1320000,2055,0.318636
3,UCYpRDnhk5H8h16jpS84uqsA,LrCCjX4i76I,https://i.ytimg.com/vi/LrCCjX4i76I/default.jpg,2022-05-25T15:56:08Z,2022.0,5.0,25.0,2.0,17.0,Fusillade au Texas : Joe Biden veut « tenir tê...,...,191.0,136.0,140.0,157.0,Le Monde,2006-03-17T14:41:41Z,310639457,1320000,2055,0.455985
4,UCYpRDnhk5H8h16jpS84uqsA,7K1mPMaIkM0,https://i.ytimg.com/vi/7K1mPMaIkM0/default.jpg,2022-05-22T12:00:04Z,2022.0,5.0,22.0,6.0,14.0,Ukraine-Russie : la guerre peut-elle provoquer...,...,162.0,118.0,132.0,136.0,Le Monde,2006-03-17T14:41:41Z,310639457,1320000,2055,0.850076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11550,UCewhc0fvja891XkpIPGRMxQ,KMJKo-Gp_5A,https://i.ytimg.com/vi/KMJKo-Gp_5A/default.jpg,2022-01-28T18:17:32Z,2022.0,1.0,28.0,4.0,20.0,Le sosie de Michel Blanquer,...,210.0,157.0,178.0,184.0,LCI,2015-06-17T14:23:07Z,164788340,507000,3416,0.014201
11551,UCewhc0fvja891XkpIPGRMxQ,R5wgH9JYOMc,https://i.ytimg.com/vi/R5wgH9JYOMc/default.jpg,2022-01-27T09:37:10Z,2022.0,1.0,27.0,3.0,11.0,"Droite vs Gauche, le débat des jeunes ! Élysée...",...,218.0,162.0,157.0,198.0,LCI,2015-06-17T14:23:07Z,164788340,507000,3416,1.220316
11552,UCewhc0fvja891XkpIPGRMxQ,XIrbpycPqCc,https://i.ytimg.com/vi/XIrbpycPqCc/default.jpg,2022-01-27T08:01:39Z,2022.0,1.0,27.0,3.0,10.0,"""On soignera les anti-vax, les non-vaccinés, l...",...,206.0,162.0,184.0,225.0,LCI,2015-06-17T14:23:07Z,164788340,507000,3416,0.085010
11553,UCewhc0fvja891XkpIPGRMxQ,4qeaKO9zJGs,https://i.ytimg.com/vi/4qeaKO9zJGs/default.jpg,2022-01-26T10:15:48Z,2022.0,1.0,26.0,2.0,12.0,"Darmanin : ""Le Pen est la plus dangereuse pour...",...,135.0,119.0,159.0,186.0,LCI,2015-06-17T14:23:07Z,164788340,507000,3416,0.088560


In [21]:
videos = videos_m.drop(columns=['color_1_r','color_1_g','color_1_b', 'color_2_r','color_2_g','color_2_b',
                               'color_3_r','color_3_g','color_3_b', 'publishedAt_y','viewCount_channel',
                               'subscriberCount','videoCount'])

In [54]:
duration_sec = []
for elt in videos.duration.to_list():
    duration_sec.append(isodate.parse_duration(elt).total_seconds())
    

In [59]:
import isodate
videos = videos.assign(
    duration = duration_sec
)
videos.duration = videos.duration.astype(int)

In [76]:
l = []
for i in tqdm(videos.index):
    l.append(extract_emojis(removing_numbers(videos.iloc[i]['title_video'])))
videos['nb_emojis_title'] = l
l = []
for i in tqdm(videos.index):
    l.append(len((videos.iloc[i]['title_video'])))

videos['nb_words'] = l
l = []
for i in tqdm(videos.index):
    l.append(number_punctuation(remove_emojis(videos.iloc[i]['title_video'])))
videos['nb_punct_title'] = l
l = []
for i in tqdm(videos.index):
    l.append(sum(1 for c in videos.iloc[i]['title_video'] if c.isupper()))
videos['nb_upper_letter'] = l


videos["emojis_prop_title"] = videos['nb_emojis_title'] / videos['nb_words_title']
videos["punct_prop_title"] = videos['nb_punct_title'] / videos['nb_words_title']

100%|██████████████████████████████████| 11555/11555 [00:00<00:00, 14505.24it/s]


In [77]:
filepath = Path('../csv/videos2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
videos.to_csv(filepath,sep=';') 